# Extracting IMERG images

In [2]:

import h5py 
import tifffile as tff 
import numpy as np
import netCDF4 
# import imageio 


In [3]:
date = 20210324
fp = r'd:\DatasetAUS\IMERG-LD6\3B-DAY-L.MS.MRG.3IMERG.'+str(date)+'-S000000-E235959.V06.nc4.nc4'
nc = netCDF4.Dataset(fp)
image = np.array(nc['precipitationCal'])[0]
image = np.rot90(image)
outfolder = r'D:\Projects\precipitation_AUS\IMAGES\IMERG/'

tff.imwrite( outfolder + str(date) + '.tif'  , image)

In [36]:
np.array([image, image ,image ]).shape

(3, 371, 500)

In [8]:
nc.variables.keys()

dict_keys(['precipitationCal', 'HQprecipitation', 'lat', 'lon', 'time'])

In [14]:
np.max(nc['lat'] ) , np.min( nc['lat'])


(-8.049998, -45.05)

In [15]:
np.max(nc['lon'] ) , np.min( nc['lon']) 


(159.95, 110.05)

# ERA5

In [30]:

date = '2021-03-18'
fp = r'd:\DatasetAUS\ERA5\cape\2021-01-01-2021-12-31-cape.hdf5'
hdf = h5py.File(fp)
image = np.array(hdf[date])
outfolder = r'D:\Projects\precipitation_AUS\IMAGES/ERA5_tp/ERA_cape_' 

tff.imwrite( outfolder + str(date) + '.tif'  , image)


In [19]:
image.shape

(169, 281)

In [10]:
np.max(hdf['lat'] ) , np.min( hdf['lat'])


(-8.0, -50.0)

In [11]:
np.max(hdf['lon'] ) , np.min( hdf['lon']) 

(170.0, 100.0)

# PDIR to datframe

In [2]:
from osgeo import gdal
import pandas as pd

def pixel( file,x,y):
    number_of_col = file.RasterXSize 
    number_of_rows= file.RasterYSize
    left = file.GetGeoTransform()[0]

    top = file.GetGeoTransform()[3]
    res_x = file.GetGeoTransform()[1]
    res_y = file.GetGeoTransform()[5]
    J = int((x-left)/res_x)
    I = int((y-top)/res_y)
    if I>number_of_rows or J>number_of_col or I<0 or J<0 :
        print( "(",I,J ,")" ,"  is out of raster bands" )
    else:
        return I,J
    


def Dict_to_hdf(Dict , of ):
    HDF = h5py.File(of, 'w')
    for k in Dict :    
        test = HDF.create_dataset( k , data = Dict[k] )
    HDF.close()   

In [6]:
import tifffile as tff
import numpy as np
import h5py 

fp = r"D:\DatasetAUS\PDIR 2013-2024\PDIR_1d20220223.tif"
PDIR_image = gdal.Open(fp)
top = PDIR_image.GetGeoTransform()[3]
res_x = PDIR_image.GetGeoTransform()[1]
res_y = PDIR_image.GetGeoTransform()[5]
left = PDIR_image.GetGeoTransform()[0]
arr  = PDIR_image.ReadAsArray(0,0,PDIR_image.RasterXSize, PDIR_image.RasterYSize)

point_1 = ( 147.56  , -24.81 )
point_2 = ( 156  , -37.33 )

cell_1 = pixel( file= PDIR_image,x =point_1[0] ,y= point_1[1])
cell_2 = pixel( file= PDIR_image,x =point_2[0] ,y= point_2[1])
image = arr[  cell_1[0] : cell_2[0]   ,  cell_1[1] : cell_2[1] ]

tff.imwrite( r"D:\Projects\precipitation_AUS\IMAGES\pdir\PDIR_23-feb-2022-crop.tif" , image)

lon =  np.arange( point_1[0] ,   point_2[0]  ,  res_x ) 
lat =  np.arange( point_1[1] ,   point_2[1]   ,  res_y) 
lat_image = np.array( [lat for i in range(image.shape[1])]).T
lon_image = np.array( [lon for i in range(image.shape[0])])

df = {}
# df['PDIR'] = np.reshape( image ,   image.shape[0]  * image.shape[1])
df['LATITUDE'] = np.reshape( lat_image ,   image.shape[0]  * image.shape[1])
df['LONGITUDE'] = np.reshape( lon_image ,   image.shape[0]  * image.shape[1])
df['YEAR']   =  np.zeros( image.shape[0]  * image.shape[1] ) +  2022  
df['MONTH']   =  np.zeros( image.shape[0]  * image.shape[1] ) +  2  
df['DAY']   =  np.zeros( image.shape[0]  * image.shape[1] ) +  23  
Dict_to_hdf(df , r'D:\Projects\precipitation_AUS\dataset\23feb2022/data_23_feb_2022.hdf5' )


c:\Users\S4055367\AppData\Local\anaconda3\envs\Reza2\Lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.14.3 when it was built against 1.14.2, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [66]:
cell_1 , cell_2

((439, 1000), (752, 1211))

array([2, 2, 2, ..., 0, 0, 0], dtype=int16)

In [70]:
image.shape

(313, 211)

In [45]:
# import packages
import netCDF4
import pandas as pd
import numpy as np
import os 
import h5py
from datetime import datetime as Time
from datetime import timedelta as Delta



def Dict_to_hdf(Dict , of ):
    HDF = h5py.File(of, 'w')
    for k in Dict :    
        test = HDF.create_dataset( k , data = Dict[k] )
    HDF.close()   

In [43]:
# input folder and output hdf files
folder = r"D:\Projects\Morteza\New folder/"
output_file = r"D:/Projects/Morteza/grace.hdf5"


In [46]:
files = os.listdir(folder)
out_dict = {}
for i in range( len(files)):
    fp = folder + files[i]
    nc = netCDF4.Dataset(fp)
    if i == 0 :
        Shape = np.array(nc['gws_inst'])[0].shape 
    arr = np.reshape( np.array(nc['gws_inst'])[0] , Shape[0]  * Shape[1] )

    T0 = Time( year= 2003 , month= 2 , day = 2 )
    dt = Delta(days= np.array(nc['time'])[0])
    time = T0 + dt 
    out_dict[str(time) ] = arr




out_dict['shape'] = Shape

lat  = np.array(nc['lat'])
lon  = np.array(nc['lon'])

lat_image = np.array( [lat  ]* Shape[1] ).T
lon_image = np.array( [lon  ]* Shape[0] )

out_dict['lat'] =  np.reshape( lat_image, Shape[0]  * Shape[1] )
out_dict['lon'] =  np.reshape(lon_image , Shape[0]  * Shape[1] )


Dict_to_hdf(out_dict , output_file)

In [17]:
np.array( [np.array(nc['lat']).T  ]* Shape[1] ).shape

(227, 137)

In [33]:
lat_image.shape  =

In [41]:
lon_image.shape

(137, 227)

In [42]:
Shape

(137, 227)

In [36]:
lon_image 

array([[23.375, 23.625, 23.875, ..., 79.375, 79.625, 79.875],
       [23.375, 23.625, 23.875, ..., 79.375, 79.625, 79.875],
       [23.375, 23.625, 23.875, ..., 79.375, 79.625, 79.875],
       ...,
       [23.375, 23.625, 23.875, ..., 79.375, 79.625, 79.875],
       [23.375, 23.625, 23.875, ..., 79.375, 79.625, 79.875],
       [23.375, 23.625, 23.875, ..., 79.375, 79.625, 79.875]],
      dtype=float32)

In [37]:
lon_image.shape

(137, 227)

In [9]:
np.array(nc['gws_inst'])[0] 

array([[ 4.6153846,  4.6153846,  4.6153846, ...,  2.9861948,  1.2035277,
         1.5384616],
       [ 4.6153846,  4.6153846,  3.2258286, ...,  1.5384616,  1.3972311,
         1.0989012],
       [ 5.7242165,  7.9371967,  6.1934752, ...,  1.5384616,  1.5384616,
         1.0989012],
       ...,
       [ 1.5384616, 64.492226 , 58.46154  , ..., 53.214222 , 48.79121  ,
        72.4178   ],
       [64.25436  , 67.14332  , 54.829872 , ..., 86.93628  , 83.39985  ,
        79.10579  ],
       [81.870316 , 70.27741  , 51.997116 , ..., 79.56037  , 71.40202  ,
        61.487053 ]], dtype=float32)

In [11]:
out_dict['lat'] = np.array(nc['lat']) 


(137, 227)

In [12]:
np.array(nc['lat']).shape

(137,)

In [8]:
Folder  = '"er5 resample.ipynb"'



(227,)

In [14]:
arr

array([[ 4.6153846,  4.6153846,  4.6153846, ...,  2.9861948,  1.2035277,
         1.5384616],
       [ 4.6153846,  4.6153846,  3.2258286, ...,  1.5384616,  1.3972311,
         1.0989012],
       [ 5.7242165,  7.9371967,  6.1934752, ...,  1.5384616,  1.5384616,
         1.0989012],
       ...,
       [ 1.5384616, 64.492226 , 58.46154  , ..., 53.214222 , 48.79121  ,
        72.4178   ],
       [64.25436  , 67.14332  , 54.829872 , ..., 86.93628  , 83.39985  ,
        79.10579  ],
       [81.870316 , 70.27741  , 51.997116 , ..., 79.56037  , 71.40202  ,
        61.487053 ]], dtype=float32)

,PDIR,LATITUDE,LONGITUDE
0,2,-24.81,147.56
1,2,-24.81,147.60
2,2,-24.81,147.64
3,2,-24.81,147.68
4,1,-24.81,147.72
...,...,...,...
66038,0,-37.29,155.80
66039,0,-37.29,155.84
66040,0,-37.29,155.88
66041,0,-37.29,155.92
